In [ ]:
from astropy.io import ascii, fits
from astropy.table import Table, vstack
import numpy as np
import matplotlib.pyplot as plt
import es_gen
import pdb
import glob
from scipy.interpolate import interp1d
from astropy.convolution import Gaussian1DKernel, convolve
import os
import es_gen
from sp_engine import subaruSpec, measuredArray, stellModel
from specutils.io import read_fits
from astropy.time import Time

%matplotlib inline

## Originally, don't think H-alpha was covered

In [ ]:
s = subaruSpec('kic1255_fits/w111345.fits',columnUse="RawFlux")

In [ ]:
plt.plot(s.wave,s.normFlux)
plt.ylim(0,2)
plt.xlim(6560,6590)
plt.axvline(6563.,color='red')

## In a re-analysis, Kento did recover the H-alpha
Collect all red spectra and combine

In [ ]:
fList = glob.glob('new_k1255_fits_w_halpha/w*.fits')
redCCDList = []
for oneFile in fList:
    s = subaruSpec(oneFile)
    if s.ccdSide == 'red':
        redCCDList.append(oneFile)

In [ ]:
nFile = len(redCCDList)
timeArray, headArray = [], []
for ind,oneFile in enumerate(redCCDList):
    baseName = os.path.basename(oneFile)
    #s = subaruSpec(oneFile,columnUse='Flux') ## this cleaned flux appears to have artifacts
    s = subaruSpec(oneFile,columnUse='RawFlux')
    path1 = '../hirano/kic1255_20150827/{}'.format(baseName)
    path2 = '../hirano/kic1255_20150828/{}'.format(baseName)
    if os.path.exists(path1):
        head = fits.getheader(path1)
    elif os.path.exists(path2):
        head = fits.getheader(path2)
    else:
        print('No original header found for {}'.format(basename))
        head = {'DATE-OBS':"NAN",'UT-STR':"NAN"}
    timeArray.append("{}T{}".format(head['DATE-OBS'],head['UT-STR']))
    headArray.append(head)
    
    if ind == 0:
        ## Choose the wavelength grid the first time through
        pts = (s.wave > 6559.4) & (s.wave < 6570.)
        waveGrid = s.wave[pts]
        allFlux = np.zeros([nFile,len(waveGrid)])
        ptsForNormalization = (waveGrid > 6558.) & (waveGrid < 6560.)
        
    f = interp1d(s.wave,s.normFlux)
    interpolatedFlux = f(waveGrid)
    allFlux[ind,:] = interpolatedFlux/np.median(interpolatedFlux[ptsForNormalization])
    
    #print("File {}, Median={}".format(oneFile,np.median(allFlux[:,ind])))

In [ ]:
medSpec = np.nanmedian(allFlux,axis=0)
medSpecObj = measuredArray(waveGrid,medSpec,Res=6e4)
medSpecObj.removePoly()
## Get Errors from the standard deviation
stdSpec = np.nanstd(allFlux,axis=0)
medSNR = np.median(medSpec/stdSpec)
medSNR

In [ ]:
plt.plot(waveGrid,medSpec)

Put the time in the ApJ format (or what I think is the ApJ format they asked for in previous papers)

In [ ]:
t = Time(timeArray,out_subfmt='')
months =['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
tLabels = []
for oneVal in timeArray:
    oneMonth = months[int(oneVal[5:7])-1]
    tLabel = "{} {} {}, {}".format(oneVal[0:4],oneMonth,oneVal[8:10],oneVal[11:16])
    tLabels.append(tLabel)

In [ ]:
fig, ax = plt.subplots(figsize=(10,4))
for ind,oneRow in enumerate(allFlux):
    ax.plot(waveGrid,oneRow - 0.1 * ind,label=tLabels[ind])
ax.set_xlabel("Wavelength ($\AA$)")
ax.set_ylabel("Normalized Flux")
ax.legend()
ax.set_xlim(6557,6566)
ax.set_ylim(-0.7,1.5)
fig.savefig("plots/h_alpha_subaru.pdf")

### Make IRAF-friendly files for Johanna's analysis

In [ ]:
keepHeaders = ascii.read('input/header_keyword_to_keep.txt',data_start=0)

In [ ]:
for ind,oneFile in enumerate(redCCDList):
    baseName = os.path.basename(oneFile)
    s = subaruSpec(oneFile)
    
    ## Choose the wavelength subset the first time through
    pts = (s.wave > 6540.) & (s.wave < 6570.)
    
    tOut = Table()
    tOut['Wave'] = s.wave[pts]
    tOut['Flux'] = s.normFlux[pts]
    #tOut.write('output/iraf_friendly/{}'.format(baseName),
    #          overwrite=True)
    binTable = fits.BinTableHDU(tOut)
    ## Copy the header info
    for oneTag in keepHeaders:
        oneKey = oneTag[0]
        binTable.header[oneKey] = head[oneKey]
    
    binTable.writeto('output/iraf_friendly/{}'.format(baseName),
                     overwrite=True)

### Just Double Check Two of the ASCII files
Why are they so similar for 2015 Aug 28?

In [ ]:
#files=['w111549','w111553']
files=['w111345','w111349']
datArray = []
for oneFile in files:
    dat = ascii.read('new_k1255_ascii_w_halpha/{}.txt'.format(oneFile),
                     names=['wave','flat flux','flux'])
    datArray.append(dat)
#dat2 = ascii.read('new_k1255_ascii_w_halpha/w111349.txt',
#                  names=['wave','flat flux','flux'])
# dat = ascii.read('new_k1255_ascii_2nd_download/w111345.txt',
#                 names=['wave','flat flux','flux'])
# dat2 = ascii.read('new_k1255_ascii_2nd_download/w111349.txt',
#                   names=['wave','flat flux','flux'])



Indeed, the H-alpha flux looks like there's a repeat of the same spectrum? 

In [ ]:
fig, ax = plt.subplots(figsize=(15,6))
plotThickness=[3,1]

fluxArray = []
for oneInd,oneDat in enumerate(datArray):
    pts = (oneDat['wave'] > 6557) & (oneDat['wave'] < 6566)
    fluxArray.append(oneDat['flat flux'][pts])
    ax.plot(oneDat['wave'][pts],oneDat['flat flux'][pts],'.',
            linewidth=plotThickness[oneInd],
            markersize=plotThickness[oneInd] * 3,
            label=files[oneInd])
ax.legend()
ax.set_xlabel(r'Wavelength ($\AA$)')
ax.set_ylabel('Flux')
ax.set_ylim(0,1500)
fig.savefig('plots/spectra_check.pdf')

In [ ]:
s = subaruSpec('new_k1255_fits_w_halpha/w111345.fits')
s2 = subaruSpec('new_k1255_fits_w_halpha/w111345.fits',columnUse='RawFlux')
s.normalize(region=[6558,6566])
s2.normalize(region=[6558,6566])

Also, there is an apparent emission in the line's core, but this comes from the data cleaning process to make the "flat flux" for RV purposes.

In [ ]:
pts = (s.wave > 6557) & (s.wave < 6566)
plt.plot(s.wave[pts],s.normFlux[pts],label='Cleaned Flux')
plt.plot(s2.wave[pts],s2.normFlux[pts],label='Raw Flux')
plt.legend()
